In [1]:
import sys
sys.path+= ['/Users/ericxia/Documents/Research/iRF/irf/demo', '/Users/ericxia/anaconda/lib/python36.zip', '/Users/ericxia/anaconda/lib/python3.6', '/Users/ericxia/anaconda/lib/python3.6/lib-dynload', '/Users/ericxia/anaconda/lib/python3.6/site-packages', '/Users/ericxia/Documents/Research/iRF/irf']

In [1]:
import irf
from sklearn.datasets import load_boston, load_breast_cancer
import irf.ensemble.wrf as wrf
from irf.ensemble import RandomForestClassifier
from irf.ensemble.wrf import wrf_reg
import numpy as np

from sklearn.model_selection import train_test_split

from irf import irf_utils
from irf import irf_jupyter_utils

/Users/ericxia/anaconda/lib/python3.6/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)
/Users/ericxia/anaconda/lib/python3.6/site-packages/sklearn/externals/six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)


In [3]:
sys.path

['/Users/ericxia/anaconda/lib/python36.zip',
 '/Users/ericxia/anaconda/lib/python3.6',
 '/Users/ericxia/anaconda/lib/python3.6/lib-dynload',
 '',
 '/Users/ericxia/anaconda/lib/python3.6/site-packages',
 '/Users/ericxia/Documents/Research/iRF/irf',
 '/Users/ericxia/anaconda/lib/python3.6/site-packages/IPython/extensions',
 '/Users/ericxia/.ipython',
 '/Users/ericxia/Documents/Research/iRF/irf/demo',
 '/Users/ericxia/anaconda/lib/python36.zip',
 '/Users/ericxia/anaconda/lib/python3.6',
 '/Users/ericxia/anaconda/lib/python3.6/lib-dynload',
 '/Users/ericxia/anaconda/lib/python3.6/site-packages',
 '/Users/ericxia/Documents/Research/iRF/irf']

In [3]:
from pyspark.ml.fpm import FPGrowth
from pyspark.sql import SparkSession
from pyspark.sql.types import *

In [2]:
data = load_breast_cancer()
X = data['data']
y = data['target']

In [3]:
X_train, X_test, y_train, y_test = train_test_split(
        data.data, data.target, train_size=0.9)

In [7]:
rf = wrf()
weight = np.ones((X.shape[1],)) / X.shape[1]
rf.fit(X, y, feature_weight=weight)
print(rf.feature_importances_)

[0.00594245 0.01255127 0.         0.         0.         0.
 0.         0.25097326 0.         0.         0.         0.
 0.         0.         0.         0.00147196 0.         0.
 0.         0.         0.02123834 0.00541967 0.33731042 0.05473207
 0.00514887 0.         0.00098646 0.30422522 0.         0.        ]


In [8]:
all_rf_tree_data = irf_utils.get_rf_tree_data(rf=rf, X_train=X_train, X_test=X_test, y_test=y_test)

TypeError: input needs to be a recognizable object,         you have input a <class 'irf.ensemble.wrf.wrf'> object

In [ ]:
rf

In [ ]:
def convert(leaf_node_paths, feature_paths):
    leafnode_to_features = {}
    for i in range(len(leaf_node_paths)):
        leaf_node = leaf_node_paths[i][-1]
        leafnode_to_features[leaf_node] = feature_paths[i]
    return leafnode_to_features

In [ ]:
interactions = np.array([])
inp = X_train[y_train==1]

for i in range(len(rf.estimators_)):
    key = "dtree" + str(i)
    tree_dict = all_rf_tree_data[key]
    #assume that ith entry of feature_paths corresponds to ith entry of leaf_node_paths
    feature_paths = tree_dict["all_uniq_leaf_paths_features"]
    leaf_nodes = tree_dict["all_leaf_nodes"]
    leaf_node_paths = tree_dict["all_leaf_node_paths"]
    decision_tree = rf.estimators_[i]
    
    end = decision_tree.apply(inp)
    leafnode_to_features = convert(leaf_node_paths, feature_paths)
    temp_features = np.vectorize(leafnode_to_features.get, otypes=[np.ndarray])(end)
    interactions = np.concatenate((interactions, temp_features))
    


In [ ]:
#schema = StructType([StructField("id", IntegerType(), True), 
#                     StructField("list", ArrayType(), True)])

spark = SparkSession \
    .builder \
    .appName("iterative Random Forests with FP-Growth") \
    .getOrCreate()
    
input_list = [(i, interactions[i].tolist()) for i in range(len(interactions))]

df = spark.createDataFrame(input_list, ["id", "items"])
df.show()

In [ ]:
fpGrowth = FPGrowth(itemsCol="items", minSupport=0.1, minConfidence=0.6)
model = fpGrowth.fit(df)

model.freqItemsets.show(40, False)

In [ ]:
inter = [interactions[i].tolist() for i in range(len(interactions))]

In [ ]:
#If we were used to do GraphLab
import turicreate as tc
ids = tc.SArray(list(range(len(inter))))
vals = tc.SArray(inter)
sf = tc.SFrame({'id':ids,'val':vals})
model = tc.pattern_mining.create(sf, 'val',
                features=['val'], min_length = 5)

In [ ]:
from prefixspan import PrefixSpan
ps = PrefixSpan(inter)
ps.topk(100)

In [ ]:
####WILL KILL THE KERNEL FOR SOME REASON
stability_score = irf_utils.run_iRF(X_train=X_train,
                                    X_test=X_test,
                                    y_train=y_train,
                                    y_test=y_test,
                                    rf = rf)

In [ ]:
import pyfpgrowth

In [ ]:
fp_tree = pyfpgrowth.FPTree(inter, 0.9, None, None)

In [ ]:
children = fp_tree.root.children

In [ ]:
children[0].value

In [ ]:
#Go through signed iRF, check if it works with RIT code
#Make sure the code currently works with the signed iRF
#Play around with get_rf_tree_data method in iRF_utils to add signed support

In [4]:
stability_score = irf_utils.run_iRF_FPGrowth(X_train=X_train,
                                    X_test=X_test,
                                    y_train=y_train,
                                    y_test=y_test,
                                    K=5,
                                    rf=RandomForestClassifier(n_estimators=40))

In [11]:
stability_score.freqItemsets.show(20, False)

+------------+----+
|items       |freq|
+------------+----+
|[20]        |116 |
|[20, 22]    |65  |
|[20, 22, 23]|40  |
|[20, 22, 27]|54  |
|[20, 23]    |63  |
|[20, 23, 27]|51  |
|[20, 7]     |73  |
|[20, 7, 27] |54  |
|[20, 27]    |97  |
|[27]        |319 |
|[21]        |96  |
|[21, 22]    |57  |
|[21, 22, 27]|47  |
|[21, 23]    |65  |
|[21, 23, 27]|54  |
|[21, 7]     |51  |
|[21, 27]    |77  |
|[23]        |252 |
|[23, 27]    |215 |
|[22]        |247 |
+------------+----+
only showing top 20 rows



In [7]:
X_train.shape

(512, 30)

In [4]:
stability_score = irf_utils.run_iRF(X_train=X_train,
                                    X_test=X_test,
                                    y_train=y_train,
                                    y_test=y_test,
                                    K=5,
                                    rf=RandomForestClassifier(n_estimators=40),
                                    signed=True)

IndexError: index 27 is out of bounds for axis 0 with size 23